# ⏱️ Desetiminutovka – opakování z minulé lekce

Vytvořte třídu `Lampa`, která:
1. V `__init__` přijme argument `barva` a uloží ho jako atribut
2. Má atribut `sviti` – na začátku `False`
3. Má metodu `prepni()` – přepne stav (`True` ↔ `False`)
4. Má metodu `__str__()` – vrátí `"{barva} lampa – svítí/nesvítí"`

> ⚠️ **Kód pod čarou neměňte!**

<details>
<summary>🔎 Ukázka výstupu (klikni pro zobrazení)</summary>

```
Modrá lampa – nesvítí
Modrá lampa – svítí
Modrá lampa – nesvítí
```

</details>


In [ ]:
# Desetiminutovka: Váš kód sem 👇


### kód níže NEMĚNIT ###
l = Lampa("Modrá")
print(l)
l.prepni()
print(l)
l.prepni()
print(l)


---

# 🔒 Zapouzdření (Encapsulation)

## Proč potřebujeme zapouzdření?

V minulé lekci jsme vytvářeli objekty s atributy, ke kterým se dalo přistupovat **přímo zvenku**:

```python
hrac = Hrac("Petr", 100, 50)
hrac.hp = -500       # ← Nikdo nám nebrání zapsat nesmyslnou hodnotu!
hrac.pozice_x = 9999 # ← Hráč „odletěl" z obrazovky
```

To je problém – **atributy nemají žádnou ochranu**. Do pozice se dostane záporné číslo, do HP nesmyslná hodnota, do jména prázdný řetězec...

### Co je zapouzdření?

Zapouzdření znamená:
1. **Skrýt** vnitřní data objektu (atributy) před okolním kódem
2. **Řídit přístup** k nim přes speciální metody (gettery/settery)
3. **Hlídat**, aby se do atributů dostaly jen **platné hodnoty**

```
BEZ zapouzdření:                    SE zapouzdřením:
┌─────────────────┐                ┌─────────────────┐
│     Hrac        │                │     Hrac        │
│  ┌───────────┐  │                │  ┌───────────┐  │
│  │ hp = 100  │←─┼── kdo chce,    │  │ _hp = 100 │  │
│  │ x = 50    │  │   ten změní    │  │ _x = 50   │  │
│  └───────────┘  │                │  └─────┬─────┘  │
│                 │                │        │        │
│                 │                │  setter hlídá:  │
│                 │                │  hp >= 0 ✅     │
│                 │                │  x v rozsahu ✅ │
└─────────────────┘                └─────────────────┘
 ❌ Kdokoli může rozbít            ✅ Data jsou chráněná
```

> 🎯 **Zapouzdření** = ochrana dat objektu. Okolní kód pracuje s objektem jen přes „kontrolovaný vstup".


---

## ⚠️ Problém – ukázka bez ošetření

Podívejme se, co se stane, když atributy necháme úplně nechráněné:


In [ ]:
# === Bez ošetření – pozice může být záporná ===
class Hrac:
    def __init__(self, nazev, x, y):
        self.nazev = nazev
        self.x = x
        self.y = y

    def __str__(self):
        return f"Hráč {self.nazev} na pozici [{self.x}, {self.y}]"

h = Hrac("Žofka", 5, 10)
print(h)

h.x = -500     # ← nic nám nebrání!
h.y = -9999
print(h)       # Hráč je mimo obrazovku 😱


### Řešení 1: Ošetření v konstruktoru

První pokus – ošetříme hodnoty **při vytváření** objektu:


In [ ]:
# === Ošetření v konstruktoru ===
class Hrac:
    def __init__(self, nazev, x, y):
        self.nazev = nazev
        self.x = max(x, 0)    # minimum 0
        self.y = max(y, 0)

    def __str__(self):
        return f"Hráč {self.nazev} na pozici [{self.x}, {self.y}]"

h = Hrac("Žofka", 5, -15)
print(h)           # y = 0 ✅ (ošetřeno při vytváření)

h.x = -25          # ← ale TADY ošetření NEFUNGUJE!
print(h)           # x = -25 😱


> ⚠️ **Problém:** Ošetření v `__init__()` chrání pouze **moment vytváření** objektu. Pozdější změna `h.x = -25` projde bez kontroly.

Potřebujeme nástroj, který zkontroluje hodnotu **pokaždé**, když se do atributu zapisuje – a to je **setter**.


---

## 👁️ Úrovně viditelnosti atributů

Python rozlišuje tři úrovně viditelnosti podle počtu **podtržítek** před názvem:

| Zápis | Název | Viditelnost | Příklad |
|:---|:---|:---|:---|
| `self.x` | Veřejný (public) | Viditelný **odkudkoli** | `self.nazev = "Petr"` |
| `self._x` | Chráněný (protected) | Konvence: „neměňte zvenku" | `self._hp = 100` |
| `self.__x` | Soukromý (private) | **Skrytý** – nelze přímo zvenku | `self.__heslo = "abc"` |

### Jak fungují podtržítka?

```python
class Ukazka:
    def __init__(self):
        self.verejna = "vidím"       # přístupná odkudkoli
        self._chranena = "varování"  # přístupná, ale konvence říká: „neměň mě"
        self.__soukroma = "tajná"    # Python ji PŘEJMENUJE (name mangling)
```


In [ ]:
# === Tři úrovně viditelnosti ===
class Ukazka:
    def __init__(self):
        self.verejna = "vidím"
        self._chranena = "varování"
        self.__soukroma = "tajná"

    def ukazSoukromou(self):
        return self.__soukroma    # uvnitř třídy funguje normálně

obj = Ukazka()
print(f"Veřejná:  {obj.verejna}")      # ✅ funguje
print(f"Chráněná: {obj._chranena}")    # ✅ funguje (ale neměli bychom)

try:
    print(obj.__soukroma)               # ❌ AttributeError!
except AttributeError as e:
    print(f"Soukromá: ❌ {e}")

print(f"Přes metodu: {obj.ukazSoukromou()}")   # ✅ funguje


### Jedno podtržítko vs. dvě podtržítka

| | `_atribut` | `__atribut` |
|:---|:---|:---|
| **Přístup zvenku** | Ano (ale nedoporučuje se) | Ne (Python přejmenuje) |
| **Vynucené Pythonem?** | ❌ Ne – jen konvence | ✅ Ano – name mangling |
| **Kdy použít** | Většina případů | Když opravdu chcete zamezit přístupu |

> 💡 **V praxi** se nejčastěji používá **jedno podtržítko** (`_atribut`). Dvě podtržítka jsou spíše pro pokročilé případy. My budeme používat **jedno podtržítko**.


---

## 🏷️ Property – řízený přístup k atributům

Python nabízí elegantní řešení: **property**. Vypadá jako normální atribut, ale ve skutečnosti se spouští **funkce**:

```
Bez property:                      S property:
┌──────────────────┐              ┌──────────────────┐
│ hrac.x = -5      │              │ hrac.x = -5      │
│       │          │              │       │          │
│       ▼          │              │       ▼          │
│ uloží se -5 ❌   │             │ setter:          │
│                  │              │   if hodnota < 0:│
│                  │              │     hodnota = 0  │
│                  │              │   uloží se 0 ✅  │
└──────────────────┘              └──────────────────┘
```

Property se skládá ze dvou částí:

| Část | Dekorátor | Kdy se volá | Co dělá |
|:---|:---|:---|:---|
| **Getter** | `@property` | Při **čtení** hodnoty | Vrátí hodnotu atributu |
| **Setter** | `@nazev.setter` | Při **zápisu** hodnoty | Zkontroluje a uloží hodnotu |


---

## 🔧 Getter/setter metody vs. property

V jiných jazycích (Java, C#) se pro řízení přístupu k atributům používají **metody** `getX()` a `setX()`. V Pythonu to lze také, ale je to **neelegantní**:

```python
# === „Javovský" přístup – getter/setter metody ===
class Hrac:
    def __init__(self, x):
        self._x = max(x, 0)

    def getX(self):
        return self._x

    def setX(self, hodnota):
        self._x = max(hodnota, 0)

h = Hrac(10)
h.setX(-5)            # musíme volat metodu
print(h.getX())       # musíme volat metodu
```

```python
# === Pythonový přístup – property ===
class Hrac:
    def __init__(self, x):
        self.x = x            # volá setter automaticky

    @property
    def x(self):
        return self._x

    @x.setter
    def x(self, hodnota):
        self._x = max(hodnota, 0)

h = Hrac(10)
h.x = -5              # vypadá jako normální atribut
print(h.x)            # vypadá jako normální atribut
```

| | Getter/setter metody | Property |
|:---|:---|:---|
| **Zápis** | `h.setX(-5)` | `h.x = -5` |
| **Čtení** | `h.getX()` | `h.x` |
| **Čitelnost** | ❌ Nepřirozené | ✅ Přirozené |
| **Kompatibilita** | Musíme měnit kód, který třídu používá | Kód se **nemění** |

> 🎯 **V Pythonu používáme property**, ne getter/setter metody. Property vypadá jako normální atribut, ale za ním se schovává ochrana.

### Syntaxe

```python
class Hrac:
    def __init__(self, x):
        self.x = x              # ← volá SETTER (i v __init__!)

    @property                    # GETTER – čtení
    def x(self):
        return self._x           # vrátí skrytý atribut

    @x.setter                    # SETTER – zápis
    def x(self, hodnota):
        self._x = max(hodnota, 0)  # ošetření + uložení
```

```
hrac.x = -5          hrac.x
    │                    │
    ▼                    ▼
@x.setter            @property
def x(self, hodn):   def x(self):
  self._x = max(...)    return self._x
    │                    │
    ▼                    ▼
uloží 0 do _x       vrátí hodnotu _x
```

> ⚠️ **Důležité:** Skutečná data jsou uložena v `self._x` (s podtržítkem). Property `x` (bez podtržítka) je „brána", přes kterou se k datům přistupuje.


---

## 📝 Příklad: Hráč s property

Kompletní příklad – pozice hráče nemůže klesnout pod 0:


In [ ]:
# === Property – pozice hráče ===
class Hrac:
    def __init__(self, nazev, x, y):
        self.nazev = nazev
        self.x = x          # volá setter pro x
        self.y = y          # volá setter pro y

    @property
    def x(self):
        return self._x

    @x.setter
    def x(self, hodnota):
        self._x = max(hodnota, 0)    # minimum 0

    @property
    def y(self):
        return self._y

    @y.setter
    def y(self, hodnota):
        self._y = max(hodnota, 0)

    def __str__(self):
        return f"Hráč {self.nazev} na pozici [{self.x}, {self.y}]"

# === Test ===
h = Hrac("Žofka", 5, -15)
print(h)             # y = 0 ✅ (setter ošetřil už v __init__)

h.x -= 25            # snaha nastavit x = -20
print(h)             # x = 0 ✅ (setter ošetřil!)

h.x = 50
h.y = 30
print(h)             # normální hodnoty ✅


### Co přesně se děje krok za krokem?

```python
h = Hrac("Žofka", 5, -15)
```

1. Python zavolá `__init__(self, "Žofka", 5, -15)`
2. `self.nazev = "Žofka"` – normální atribut (bez property)
3. `self.x = 5` – Python vidí, že `x` má **setter** → zavolá ho → `self._x = max(5, 0)` → `_x = 5`
4. `self.y = -15` – Python vidí setter → `self._y = max(-15, 0)` → `_y = 0` ✅

```python
h.x -= 25   # = h.x = h.x - 25 = 5 - 25 = -20
```

5. Python čte `h.x` → volá **getter** → vrátí `self._x` = 5
6. Vypočítá `5 - 25 = -20`
7. Python zapisuje `h.x = -20` → volá **setter** → `self._x = max(-20, 0)` → `_x = 0` ✅

> 💡 **Klíčové:** Setter se volá **pokaždé** při zápisu – včetně `__init__()`, `+=`, `-=` i přímého přiřazení.


---

## 💡 Praktický příklad – Teplomer

Property nemusí jen hlídat rozsah – může provádět i **výpočty**:


In [ ]:
# === Teplomer – property s výpočtem ===
class Teplomer:
    def __init__(self, teplota_c):
        self.celsius = teplota_c          # volá setter

    @property
    def celsius(self):
        return self._celsius

    @celsius.setter
    def celsius(self, hodnota):
        if hodnota < -273.15:             # absolutní nula
            self._celsius = -273.15
        else:
            self._celsius = hodnota

    @property
    def fahrenheit(self):                 # POUZE getter (read-only) -> dá se představit jako self.fahrenheit = self._celsius * 9/5 + 32
        return self._celsius * 9/5 + 32

    def __str__(self):
        return f"Teploměr: {self.celsius}°C ({self.fahrenheit:.1f}°F)"

# === Test ===
t = Teplomer(25)
print(t)                # 25°C (77.0°F)

t.celsius = -300        # pod absolutní nulou
print(t)                # -273.15°C (-459.7°F)

t.celsius = 100
print(t)                # 100°C (212.0°F)
print(f"Fahrenheit: {t.fahrenheit}") #gettter pro fahrenheit, protože je property bez setteru 


> 💡 **Read-only property:** Vlastnost `fahrenheit` má **pouze getter** (žádný setter). Nelze do ní přímo zapsat:
>
> ```python
> t.fahrenheit = 50   # ❌ AttributeError: can't set attribute
> ```
>
> To je užitečné pro **vypočítané hodnoty**, které nemá smysl nastavovat přímo.


### 🧩 Property nemusí mít atribut v `__init__()`

Všimněte si, že `fahrenheit` **nikde v `__init__()` neukládáme** do `self._fahrenheit`. Přesto funguje:

```
 __init__(self, teplota_c):
 │
 └──▶ self.celsius = teplota_c     # uloží jen _celsius
```

```
 @property
 def fahrenheit(self):
 │
 └──▶ return self._celsius * 9/5 + 32   # pokaždé VYPOČÍTÁ z _celsius
```

Proč to funguje?
- **Property getter** se spustí **pokaždé**, když k atributu přistoupíme
- Nepotřebuje žádný `self._fahrenheit` – **počítá hodnotu za běhu** z jiného atributu
- V paměti existuje pouze `_celsius`, `fahrenheit` je jen **„průhled"** na něj

> 🎯 **Property = nemusí ukládat vlastní data.** Může pouze počítat, filtrovat nebo transformovat jiné atributy. Díky tomu se hodí i pro odvozené hodnoty jako procenta, formátování nebo převody jednotek.

---

## 🔄 Před a po – srovnání

| | Bez property | S property |
|:---|:---|:---|
| **Zápis** | `self.x = hodnota` | `self.x = hodnota` (stejný!) |
| **Čtení** | `print(obj.x)` | `print(obj.x)` (stejný!) |
| **Ochrana** | ❌ Žádná | ✅ Setter kontroluje |
| **Data uložena v** | `self.x` | `self._x` (skrytý atribut) |

> 🎯 **Hlavní výhoda property:** Kód, který třídu **používá**, se **nezmění**. Stále píšeme `hrac.x = 5`. Ale nyní je za tím ochrana, o které uživatel třídy ani nemusí vědět.


---

## ❓ Kdy použít property?

| Situace | Příklad | Řešení |
|:---|:---|:---|
| Atribut má **omezený rozsah** | HP: 0–100, pozice: 0–šířka | Setter s `min()`/`max()` |
| Atribut závisí na **jiném atributu** | Fahrenheit z Celsia | Getter s výpočtem |
| Chcete **logovat** změny | Zaznamenat kdo co změnil | Setter s `print()` |
| Atribut je **read-only** | Plocha obdélníku z a × b | Pouze getter |
| Atribut **nemá omezení** | Jméno, barva... | Property **NEPOTŘEBUJETE** |

> 💡 **Nepoužívejte property zbytečně!** Pokud atribut nemá žádná omezení, nechte ho jako normální veřejný atribut.


---

# ✏️ Cvičení


## Cvičení 1 – Skrytý atribut

Vytvořte třídu `Zeton`, která má **skrytý** (private) atribut `hodnota` (s dvěma podtržítky).

Při vytváření se předá hodnota žetonu a uloží se do skrytého atributu.

V tomto cvičení **nemusíte** vytvářet property – jen skryjte atribut.

> ⚠️ **Kód pod čarou neměňte!**

<details>
<summary>🔎 Očekávaný výstup (klikni pro zobrazení)</summary>

```
V pořádku
```

</details>


In [ ]:
# Cvičení 1: Váš kód sem 👇


### kód níže NEMĚNIT ###
try:
    Z1 = Zeton(10)
    expected_var = "hodnota"
    all_vars = dir(Z1)
    if "_Zeton__" + expected_var in all_vars:
        print("V pořádku")
    elif expected_var in all_vars:
        print(expected_var, "není skrytý atribut")
    elif "_" + expected_var in all_vars:
        print("_" + expected_var, "není skrytý atribut")
    else:
        print(expected_var, "nenalezeno")
except NameError:
    print("Třída Zeton nenalezena")
except TypeError:
    print("Třída Zeton má přijímat jeden argument při vytváření")


---

## Cvičení 2 – Property s ošetřením

Upravte třídu `Zeton` tak, aby měla **property** `hodnota`:
1. **Getter** – vrátí hodnotu žetonu
2. **Setter** – minimální hodnota žetonu je `50`. Pokud se pokusíme nastavit méně, uloží se `50`.

> ⚠️ **Kód pod čarou neměňte!**

<details>
<summary>🔎 Očekávaný výstup (klikni pro zobrazení)</summary>

```
V pořádku
```

</details>


In [ ]:
# Cvičení 2: Váš kód sem 👇


### kód níže NEMĚNIT ###
#_____________________________________________________________________
Z1 = Zeton(10)
if Z1.hodnota != 50:
    print(f"Při vytváření se dostala hodnota {Z1.hodnota} (očekáváno 50)")
else:
    Z1.hodnota = 20
    if Z1.hodnota != 50:
        print(f"Při úpravě se dostala hodnota {Z1.hodnota} (očekáváno 50)")
    else:
        Z1.hodnota = 60
        if Z1.hodnota != 60:
            print(f"Hodnota 60 je platná, ale uložilo se {Z1.hodnota}")
        else:
            print("V pořádku")


---

## Cvičení 3 – Hráč s HP

Vytvořte třídu `Postava` s:
1. Atributem `nazev` (veřejný, bez property)
2. Property `hp` – hodnota **musí být v rozsahu 0–100**
   - Pokud je menší než 0 → nastaví se `0`
   - Pokud je větší než 100 → nastaví se `100`
3. Metodou `__str__()` – vrátí `"{název}: {hp} HP"`

> ⚠️ **Kód pod čarou neměňte!**

<details>
<summary>🔎 Očekávaný výstup (klikni pro zobrazení)</summary>

```
Bojovník: 100 HP
Bojovník: 80 HP
Bojovník: 0 HP
Bojovník: 100 HP
```

</details>


In [ ]:
# Cvičení 3: Váš kód sem 👇


### kód níže NEMĚNIT ###
p = Postava("Bojovník", 100)
print(p)
p.hp -= 20
print(p)
p.hp = -50
print(p)
p.hp = 999
print(p)


---

### 🌟 Bonus – Bankovní účet se zapouzdřením

Vytvořte třídu `Ucet` s:
1. Veřejným atributem `majitel`
2. Property `zustatek`:
   - Nelze nastavit na **zápornou** hodnotu (minimum 0)
3. Metodou `vloz(castka)` – přičte k zůstatku (pouze kladné hodnoty)
4. Metodou `vyber(castka)` – odečte ze zůstatku (pokud je dostatek prostředků)
5. `__str__()` – vrátí `"Účet: {majitel} – {zustatek} Kč"`

> ⚠️ **Kód pod čarou neměňte!**

<details>
<summary>🔎 Očekávaný výstup (klikni pro zobrazení)</summary>

```
Účet: Petr – 1000 Kč
Účet: Petr – 1500 Kč
Účet: Petr – 1200 Kč
❌ Nedostatek prostředků!
Účet: Petr – 1200 Kč
```

</details>


In [ ]:
# Bonus: Váš kód sem 👇


### kód níže NEMĚNIT ###
u = Ucet("Petr", 1000)
print(u)
u.vloz(500)
print(u)
u.vyber(300)
print(u)
u.vyber(5000)
print(u)


---

## 📋 Shrnutí

### Úrovně viditelnosti

| Zápis | Název | Přístup zvenku |
|:---|:---|:---|
| `self.atribut` | Veřejný | ✅ Ano |
| `self._atribut` | Chráněný (konvence) | ⚠️ Ano, ale nemělo by se |
| `self.__atribut` | Soukromý | ❌ Ne (name mangling) |

### Property

```python
class Trida:
    def __init__(self, hodnota):
        self.atribut = hodnota       # volá setter!

    @property                         # GETTER – čtení
    def atribut(self):
        return self._atribut

    @atribut.setter                   # SETTER – zápis
    def atribut(self, hodnota):
        # ošetření hodnoty
        self._atribut = hodnota       # uložení do skrytého _atributu
```

### Pravidla

| Pravidlo | |
|:---|:---|
| Data uložte do `self._atribut` | S jedním podtržítkem |
| Property (getter/setter) pojmenujte **bez** podtržítka | `@property def atribut(self):` |
| Setter se volá i v `__init__()` | `self.atribut = hodnota` → setter |
| Setter musí obsahovat **ošetření** | Jinak nemá smysl |
| Nepoužívejte property zbytečně | Jen když je potřeba hlídat hodnoty |

> 🎯 **Zapamatuj si:** Property = „brána" k datům. Vypadá jako normální atribut, ale za ní je funkce, která hlídá správnost hodnot.
